In [43]:
import os
from rspace_client.eln import eln
import json
import openai
import requests
from tenacity import retry, wait_random_exponential, stop_after_attempt
from termcolor import colored

eln_cli = eln.ELNClient(os.getenv("RSPACE_URL"), os.getenv("RSPACE_API_KEY"))
print(eln_cli.get_status())
GPT_MODEL = "gpt-4"

{'message': 'OK', 'rspaceVersion': '1.91.1'}


In [44]:
print(eln_cli.get_documents()['documents'][0]['name'])


Anti-aurora Western


In [39]:
docs = eln_cli.get_documents(query='l: lastModified:[2020-01-01 TO *]')['documents']
print(list(map(lambda x: f" {x['name']} - {x['created']}", docs)))

[]


In [46]:
@retry(wait=wait_random_exponential(multiplier=1, max=40), stop=stop_after_attempt(3))
def chat_completion_request(messages, functions=None, function_call=None, model=GPT_MODEL):
    headers = {
        "Content-Type": "application/json",
        "Authorization": "Bearer " + openai.api_key,
    }
    json_data = {"model": model, "messages": messages}
    if functions is not None:
        json_data.update({"functions": functions})
    if function_call is not None:
        json_data.update({"function_call": function_call})
    try:
        response = requests.post(
            "https://api.openai.com/v1/chat/completions",
            headers=headers,
            json=json_data,
        )
        return response
    except Exception as e:
        print("Unable to generate ChatCompletion response")
        print(f"Exception: {e}")
        return e

In [104]:
def search_rspace_eln(luceneQuery, sort_order="lastModified desc"):
    q = "l: " + luceneQuery
    docs = eln_cli.get_documents(query=q, order_by=sort_order)['documents']
    print(str.join('\n\n', list(map(lambda x: f" {x['name']} - {x['created']} - {x['tags']}", docs))))

In [111]:
available_functions = {
 "lucene":search_rspace_eln
}

functions = [
  {
    "name": "lucene",
    "description": """
    A valid Lucene Query Language string generated from user input.
    Document fields are name, docTag, fields.fieldData .
    Don't use wildcards
    """,
    "parameters": {
        "type":"object",
        "properties": {
            "luceneQuery": {
                "type":"string"
            },
            "sort_order": {
                "type":"string",
                "enum":["name asc", "name desc", "created asc", "created desc"]
            },
            
        }
  }
  }
]

In [115]:
messages = [
 {
     "role" : "system",
     "content": "Generate function arguments from user input. Don't show reasoning."
 },
 {
     "role" : "user",
     "content": "Find documents that are tagged with 'polyclonal', or contains 'Ben' (but not wyvis) in the text. List results in reverse alphabetical order"
 }
    
]

In [116]:
resp = chat_completion_request(messages, functions, {'name':'lucene'})
print(resp)
response_message = resp.json()['choices'][0]['message']
print(json.dumps(response_message, indent=2))
if response_message['function_call'] is not None:
    f_name = response_message['function_call']['name']
    f_args = json.loads(response_message['function_call']['arguments'])
    result = available_functions[f_name](**f_args)

<Response [200]>
{
  "role": "assistant",
  "content": null,
  "function_call": {
    "name": "lucene",
    "arguments": "{\n  \"luceneQuery\": \"docTag:polyclonal OR (fields.fieldData:Ben NOT fields.fieldData:wyvis)\",\n  \"sort_order\": \"name desc\"\n}"
  }
}
 West Ben Lawers range - 2019-12-01T11:10:45.498Z - None

 Generation of anti-aurora antibodies - 2023-09-11T19:49:28.418Z - polyclonal

 ben vorlich (Arrochar) - 2019-12-01T10:51:13.585Z - None

 Ben Dorain, Ben an Dothaidh - 2018-12-29T21:53:39.369Z - None

 Ben Chonzie - 2018-02-18T22:54:48.925Z - munro

 Ben Challum - 2018-07-08T19:00:24.247Z - None

 Beinn Teallach, A'Chaorainn - 2018-02-18T23:00:49.638Z - munro

 Beinn a Ghlo - 2018-12-29T21:59:58.827Z - None

 Anti-aurora Western - 2023-09-11T19:43:38.318Z - Western,aurora,ECL,polyclonal
